In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.autograd as tgrad
from torch.autograd import Variable

import os
import time
import utils
import numpy as np

from tqdm import tqdm, trange
import matplotlib.pyplot as plt

import networks
import importlib

In [2]:
os.environ['KMP_DUPLICATE_LIB_OK']='True'
seed = 1234
torch.set_default_dtype(torch.float32)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(torch.cuda.is_available())
# torch.set_default_tensor_type(torch.DoubleTensor)
print(device)

if device == 'cuda': 
    print(torch.cuda.get_device_name())

True
cuda


# Data Sampling
Here in our case, the system is European Call Option PDE and the physical information about the system consists of Boundary Value conditions, final Value conditions and the PDE itself.

In [3]:
K = 40
r = 0.05
sigma = 0.25
T = 1
S_range = [0, 130]
t_range = [0, T]
gs = lambda x: np.fmax(x-K, 0)

# Build Neural Network

In [4]:
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers):
        super(Net, self).__init__()
        self.layers = nn.ModuleList([nn.Linear(input_size, hidden_size)])
        self.layers.extend([nn.Linear(hidden_size, hidden_size) for _ in range(n_layers - 1)])
        self.output = nn.Linear(hidden_size, output_size)
        self.relu = nn.ReLU()

    def forward(self, x):
        for layer in self.layers:
            x = self.relu(layer(x))
        x = self.output(x)
        return x



# layers = [2, 50, 50, 50, 50, 50, 50, 50, 50, 1] # Network structure
net = Net(2, 50,1,8).to(device) #  Network initialization

In [5]:
n_epochs = 60000
lossFunction = nn.MSELoss()

from torchimize.functions import lsq_lma
# coeffs_list = lsq_lma(fnn.parameters(), function=lossFunction)
optimizer = optim.Adam(net.parameters(), lr=0.003)

x_f_s = torch.tensor(0.).float().to(device).requires_grad_(True)
x_label_s = torch.tensor(0.).float().to(device).requires_grad_(True)
optimizer_adam_weight = torch.optim.Adam([x_f_s] + [x_label_s], lr=0.0003)


samples = {"pde": 5000, "bc":500, "fc":500}

# Modelling


In [6]:
loss_hist = []
start_time = time.time()

for epoch in range(n_epochs):
    bc_st_train, bc_v_train, n_st_train, n_v_train = \
    utils.trainingData(K, 
                       r, 
                       sigma, 
                       T, 
                       S_range[-1], 
                       S_range, 
                       t_range, 
                       gs, 
                       samples['bc'], 
                       samples['fc'], 
                       samples['pde'], 
                       RNG_key=123)
    # save training data points to tensor and send to device
    n_st_train = torch.from_numpy(n_st_train).float().requires_grad_().to(device)
    n_v_train = torch.from_numpy(n_v_train).float().to(device)
    
    bc_st_train = torch.from_numpy(bc_st_train).float().to(device)
    bc_v_train = torch.from_numpy(bc_v_train).float().to(device)


    # PDE Round
    y1_hat = net(n_st_train)
    grads = tgrad.grad(y1_hat, n_st_train, grad_outputs=torch.ones(y1_hat.shape).cuda(), retain_graph=True, create_graph=True, only_inputs=True)[0]
    # print(grads)
    dVdt, dVdS = grads[:, 0].view(-1, 1), grads[:, 1].view(-1, 1)
    grads2nd = tgrad.grad(dVdS, n_st_train, grad_outputs=torch.ones(dVdS.shape).cuda(), create_graph=True, only_inputs=True)[0]
    # print(grads2nd)
    d2VdS2 = grads2nd[:, 1].view(-1, 1)
    S1 = n_st_train[:, 1].view(-1, 1)
    pde_loss = lossFunction(-dVdt, 0.5*((sigma*S1)**2)*d2VdS2 + r*S1*dVdS - r*y1_hat)
    
    
    # conditions Round
    y21_hat = net(bc_st_train)
    bc_loss = lossFunction(bc_v_train, y21_hat)
    
    
    # Backpropagation and Update
    optimizer.zero_grad()
    total_loss = pde_loss + bc_loss
    combined_loss = torch.exp(-x_f_s.detach()) * pde_loss + torch.exp(-x_label_s.detach()) * bc_loss
    combined_loss.backward()
    optimizer.step()
    
    loss_hist.append(total_loss.item())
    if epoch % 500 == 0:
        print(f'{epoch}/{n_epochs} PDE Loss: {pde_loss.item():.5f}, BC Loss: {bc_loss.item():.5f}, nn_loss: {combined_loss.item():5f}, total loss: {total_loss.item():.5f}, minimum loss: {min(loss_hist):.5f}')
        print(f'the weight is {torch.exp(-x_f_s.detach()).item():.5f}, {torch.exp(-x_label_s.detach()).item():.5f}')
    
    # update the weight
    optimizer_adam_weight.zero_grad()
    loss = torch.exp(-x_f_s) * pde_loss.detach() + x_f_s + torch.exp(-x_label_s) * bc_loss.detach() + x_label_s
    loss.backward()
    optimizer_adam_weight.step()
    pass

end_time = time.time()

print('run time:', end_time - start_time)
print('Adam done!')

0/60000 PDE Loss: 0.00001, BC Loss: 3357.17090, nn_loss: 3357.170898, total loss: 3357.17090, minimum loss: 3357.17090
the weight is 1.00000, 1.00000
500/60000 PDE Loss: 0.00259, BC Loss: 12.00583, nn_loss: 11.898733, total loss: 12.00841, minimum loss: 0.13378
the weight is 1.15721, 0.99083
1000/60000 PDE Loss: 0.00193, BC Loss: 0.55265, nn_loss: 0.549831, total loss: 0.55458, minimum loss: 0.11225
the weight is 1.34884, 0.99018
1500/60000 PDE Loss: 0.00479, BC Loss: 0.17529, nn_loss: 0.181032, total loss: 0.18008, minimum loss: 0.10863
the weight is 1.56959, 0.98984
2000/60000 PDE Loss: 0.00425, BC Loss: 0.30080, nn_loss: 0.305410, total loss: 0.30505, minimum loss: 0.10863
the weight is 1.82502, 0.98952
2500/60000 PDE Loss: 0.00310, BC Loss: 0.11650, nn_loss: 0.121846, total loss: 0.11960, minimum loss: 0.10863
the weight is 2.12163, 0.98939
3000/60000 PDE Loss: 0.00157, BC Loss: 0.12841, nn_loss: 0.130893, total loss: 0.12998, minimum loss: 0.10863
the weight is 2.46634, 0.98917
35

KeyboardInterrupt: 